In [1]:
import time
from typing import Dict, List, Tuple, Optional
import math

# --- 1. Graph Construction ---

def build_Cnr_graph(n: int, r: int) -> Tuple[Dict[int, List[int]], List[int]]:
    """
    Builds the Adjacency List for the Circulant Graph C_{n,r} in O(n*r) time.

    The graph has n vertices {0, 1, ..., n-1}.
    Each vertex i connects to i ± j (mod n) for j = 1 to r.

    Args:
        n: The number of vertices (n >= 3).
        r: The maximum distance of neighbors on each side (1 <= r < n/2).

    Returns:
        A tuple: (Adjacency List, Fixed Vertex Order List)
    """
    if n < 3:
        raise ValueError("n must be at least 3 for a meaningful graph.")
    if r < 1 or r >= n / 2:
        raise ValueError("r must be between 1 and floor(n/2) - 1.")

    V = n
    adj: Dict[int, List[int]] = {i: [] for i in range(V)}

    # Fixed vertex ordering: 0, 1, 2, ..., n-1
    order = list(range(V))

    # Determine the connection set
    connection_set = list(range(1, r + 1))

    for u in range(V):
        for j in connection_set:
            # Neighbor 1: u + j (mod n)
            v1 = (u + j) % n
            # Neighbor 2: u - j (mod n)
            v2 = (u - j) % n

            # Add edges, ensuring no self-loops (u != v1/v2, implicitly handled by r < n/2)

            if v1 not in adj[u]:
                adj[u].append(v1)
            if u not in adj[v1]:
                adj[v1].append(u)

            if v2 not in adj[u]:
                adj[u].append(v2)
            if u not in adj[v2]:
                adj[v2].append(u)

    return adj, order

# --- 2. Labeling Algorithm (DFS Backtracking) ---

def solve_Cnr_labeling(n: int, r: int) -> Tuple[int, Optional[List[int]]]:
    """
    Finds the minimum label set size k (k=k0, k0+1, ...) that permits
    a valid labeling for C_{n,r} where:
    1. Vertex labels l(v) are unique and in {0, ..., k}.
    2. Edge weights w(u,v) = l(u) + l(v) are unique, non-zero, and <= 2k.

    Args:
        n: The number of vertices.
        r: The connection distance parameter.

    Returns:
        A tuple: (Minimum k found, List of V labels or None if failed)
    """
    try:
        adj, order = build_Cnr_graph(n, r)
    except ValueError as e:
        print(f"Error building graph: {e}")
        return -1, None

    V = n
    E = n * r # Total number of edges

    # Initial lower bound k0 = ceil(E / 2)
    k_min_theoretical = math.ceil(E / 2)
    k = k_min_theoretical

    print(f"Graph C_{n},{r}: V={V}, E={E}.")
    print(f"Starting search for minimum k (label set size) from k₀={k}...")

    # Maximum label set size to search before giving up (safety break)
    k_max_search = k_min_theoretical + V

    while k <= k_max_search:
        start_time = time.time()

        # Initialization for the current k
        labels = [-1] * V             # Stores l(v), -1 means unassigned
        usedLabel = [False] * (k + 1) # Tracks unique vertex labels used (0 to k)
        # Tracks unique edge weights used (1 to 2k). Index 0 is unused (weight=0 is forbidden)
        usedWeight = [False] * (2 * k + 1)

        print(f"\n--- Testing k = {k} ---")

        def backtrack(idx: int) -> bool:
            """
            Recursive DFS to assign label to vertex order[idx].
            """
            if idx == V:
                # Success: All vertices assigned valid labels
                return True

            u = order[idx]

            # Iterate through all available labels (0 to k)
            for current_label in range(k + 1):

                # Constraint 1: Vertex labels must be unique
                if usedLabel[current_label]:
                    continue

                ok = True
                new_weights = []

                # Constraint 2: Check edge weight uniqueness with ALREADY LABELED neighbors
                for v in adj[u]:
                    if labels[v] != -1:
                        # Neighbor 'v' is already labeled
                        w = current_label + labels[v]

                        # Constraint checks: w must be positive, <= 2k, and unused
                        if w == 0 or w > 2 * k or usedWeight[w]:
                            ok = False
                            break
                        new_weights.append(w)

                if not ok:
                    continue  # Try next label

                # --- Assignment (Pruning passed) ---
                labels[u] = current_label
                usedLabel[current_label] = True
                for w in new_weights:
                    usedWeight[w] = True

                # Recurse to next vertex
                if backtrack(idx + 1):
                    return True

                # --- Backtrack (Undo assignment) ---
                labels[u] = -1
                usedLabel[current_label] = False
                for w in new_weights:
                    usedWeight[w] = False

            return False

        # Start the recursive search
        if backtrack(0):
            runtime = time.time() - start_time
            print(f"Success! Minimum k = {k}")
            print(f"Runtime for k={k}: {runtime:.4f} seconds.")
            return k, labels

        runtime = time.time() - start_time
        print(f"Failed for k={k}. Runtime: {runtime:.4f} seconds.")

        # If failed for current k, increment k and try again
        k += 1

    print(f"\nWarning: Search reached k={k_max_search} limit without finding a solution.")
    return -1, None


# --- 3. Example Execution (Suitable for Notebook) ---

if __name__ == "__main__":

    # NOTE: Due to exponential complexity, use small graphs (V <= 10) for quick results.
    # C_5,1 is the cycle graph C_5. E=5. k0=3.
    # C_5,2 is the Petersen graph C_5,2 (the only 5-vertex Circulant graph). E=10. k0=5.

    N_TEST = 5
    R_TEST = 2

    print(f"# --- Running C_{N_TEST},{R_TEST} Labeling ---")

    try:
        min_k, result_labels = solve_Cnr_labeling(N_TEST, R_TEST)

        if result_labels is not None and min_k != -1:
            print("\n## ✨ Final Result ✨ ##")
            print(f"Graph: C_{N_TEST},{R_TEST}")
            print(f"Minimum Label Set Size k: {min_k}")
            print(f"Vertex Labels (0..{min_k}): {result_labels}")

            # Additional verification: Check vertex labels and edge weights
            adj, _ = build_Cnr_graph(N_TEST, R_TEST)
            edge_weights = set()

            for u in range(len(result_labels)):
                for v in adj[u]:
                    if u < v: # Check each edge once
                        w = result_labels[u] + result_labels[v]
                        if w in edge_weights:
                            print(f"\nERROR: Duplicate weight {w} found!")
                        edge_weights.add(w)

            E_actual = (sum(len(adj[i]) for i in adj) // 2)
            print(f"Number of Edges E: {E_actual}")
            print(f"Total Unique Edge Weights Found: {len(edge_weights)}")

        else:
            print("\nLabeling failed to find a solution or reached search limits.")

    except ValueError as e:
        print(f"\nError: {e}")

# --- Running C_5,2 Labeling ---
Graph C_5,2: V=5, E=10.
Starting search for minimum k (label set size) from k₀=5...

--- Testing k = 5 ---
Failed for k=5. Runtime: 0.0010 seconds.

--- Testing k = 6 ---
Failed for k=6. Runtime: 0.0029 seconds.

--- Testing k = 7 ---
Success! Minimum k = 7
Runtime for k=7: 0.0000 seconds.

## ✨ Final Result ✨ ##
Graph: C_5,2
Minimum Label Set Size k: 7
Vertex Labels (0..7): [0, 1, 2, 4, 7]
Number of Edges E: 10
Total Unique Edge Weights Found: 10
